In [4]:
class Data(object):
        def __init__(self):
            self._x_train = 'X_train.csv'
            self._x_test  = 'X_test.csv'
            self._y_train = 'y_train.csv'
            self._y_test  = 'y_test.csv'

        @property
        def x_train_file(self):
            return self._x_train

        @property
        def y_train_file(self):
            return self._y_train

        @property
        def x_test_file(self):
            return self._x_test

        @property
        def y_test_file(self):
            return self._y_test
        @property
        def date_column(self):
            return ['click_time']
        @property
        def test_full(self):
            return 'test_full.csv'

data= Data()

In [5]:
import pandas as pd

In [6]:
def read_data():
    X_train= pd.read_csv(data.x_train_file,parse_dates=data.date_column)
    X_test = pd.read_csv(data.x_test_file ,parse_dates=data.date_column)
    y_train= pd.read_csv(data.y_train_file)
    y_test = pd.read_csv(data.y_test_file )           
    return X_train,X_test,y_train, y_test

In [7]:
X_train_raw,X_test_raw,y_train_raw, y_test_raw = read_data()
X_train_raw.head()

,ip,app,device,os,channel,click_time,is_attributed
0,106857,19,6,29,213,2017-11-07 01:02:24,1
1,75754,19,1699,24,213,2017-11-08 06:22:48,1
2,109837,2,1,13,435,2017-11-08 04:00:23,0
3,248392,19,0,21,213,2017-11-08 15:41:55,1
4,100212,1,2,49,120,2017-11-09 08:18:11,0


In [8]:
import datetime
def datetime_to_float(a):
    d = None
    if(isinstance(a, str)):
        d=datetime.datetime.strptime(a, "%Y-%m-%d %H:%M:%S")
    elif(isinstance(a, pd.DatetimeIndex)):
        d=a
    else:
        return 0.0        
    epoch = datetime.datetime.utcfromtimestamp(0)
    total_seconds =  (d - epoch).total_seconds()
    return total_seconds

date_transformer = lambda x: datetime_to_float(x)
assert(datetime_to_float('2017-11-07 09:35:17') == 1510047317)
assert(datetime_to_float(pd.DatetimeIndex([pd.to_datetime('2017-11-07 09:35:17')])) == 1510047317)

In [9]:
def preprocess(data):
    data['click_time']=data['click_time'].map(date_transformer)
    if 'attributed_time' in data.columns:
        data['attributed_time']=data['attributed_time'].map(date_transformer)
    return data


In [10]:
X_train=preprocess(X_train_raw)
y_train=y_train_raw

In [11]:
input_dim=X_train.shape[1]

In [12]:
def get_model():
    import tensorflow as tf
    from keras.models import Sequential
    from keras.layers.core import Dense
    from keras.layers import Activation
    model = Sequential()
    model.add(Dense(100, input_dim=input_dim, activation='relu'))
    model.add(Dense(70, activation='relu'))
    model.add(Dense(35, activation='relu'))
    model.add(Dense(17, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [13]:
model = get_model()

In [70]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix,accuracy_score, roc_curve, auc, roc_auc_score

In [90]:
from keras import backend as K
import tensorflow as tf
from tensorflow import metrics

def my_auc(y_true,y_pred):
    return metrics.auc(y_true,y_pred)

In [92]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics = [my_auc])
# my_print=PrintCallBack()

AttributeError: 'tuple' object has no attribute 'dtype'

In [85]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=2,callbacks=[my_print])


Train on 2489 samples, validate on 2489 samples
Epoch 1/2
2489/2489 [==============================] - 1s 351us/step - loss: 8.2630 - cust_auc: 0.5127 - val_loss: 7.8551 - val_cust_auc: 0.4873
Epoch 2/2
2489/2489 [==============================] - 0s 124us/step - loss: 8.2630 - cust_auc: 0.5127 - val_loss: 7.8551 - val_cust_auc: 0.4873


In [67]:
print(my_print.aucs)

[0.49423564467485209, 0.49813798788981434]


In [78]:
y_valid_predic=model.predict(X_test)

In [31]:
from sklearn.metrics import f1_score

In [32]:
y_test.is_attributed.astype(np.float32).head()

0    0.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: is_attributed, dtype: float32

In [17]:
threshold= 1e-10

In [33]:
y_valid_predic[y_valid_predic<threshold] = 0

In [20]:
y_test_update = y_test.is_attributed[y_test.is_attributed<threshold] = 0

In [37]:
y_valid_pred_series=array_to_series(y_valid_predic)

In [56]:
y_valid_pred_series.value_counts()
y_test.astype(np.float32).is_attributed.value_counts()

1.000000    2488
0.999621       1
dtype: int64

0.0    1276
1.0    1213
Name: is_attributed, dtype: int64

In [46]:
assert (type(y_valid_pred_series) == type(y_test.is_attributed.astype(np.float32)))

In [44]:
type(y_valid_pred_series)
type(y_test.is_attributed.astype(np.float32))

pandas.core.series.Series

pandas.core.series.Series

In [ ]:
y_valid_pred_series

In [59]:
def series_to_np_array(a):
    return a.values.reshape((-1,1))

In [71]:
a

array([[ 1.],
       [ 1.],
       [ 1.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32)

In [66]:
np.unique(b)

array([ 0.,  1.], dtype=float32)

In [80]:
import numpy as np
from sklearn.metrics import f1_score
import pandas as pd

f1_score((y_test.is_attributed),y_valid_predic.astype(np.int32), average='micro')  

0.48774608276416231

In [ ]:
0.32766072393300921

In [48]:
f1_score(y_test.is_attributed.astype(np.float32),y_valid_pred_series,average='binary')

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [84]:
type(y_valid_pred_series[0])

numpy.float32

In [90]:
type(y_test.is_attributed.astype(float))

numpy.int64

In [27]:
def get_processed_data(X_test_raw,y_test_raw):
    return preprocess(X_test_raw),y_test_raw
X_test,y_test=get_processed_data(X_test_raw,y_test_raw)

In [56]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=2)

Train on 2489 samples, validate on 2489 samples
Epoch 1/2
2489/2489 [==============================] - 1s 219us/step - loss: 8.2630 - mcor: 0.0000e+00 - val_loss: 7.8551 - val_mcor: 0.0000e+00
Epoch 2/2
2489/2489 [==============================] - 0s 120us/step - loss: 8.2630 - mcor: 0.0000e+00 - val_loss: 7.8551 - val_mcor: 0.0000e+00


In [15]:
def evaluate(X_test_raw,y_test_raw):
    X_test=preprocess(X_test_raw)
    y_test=y_test_raw
    return model.evaluate(X_test, y_test, verbose=0)

evaluate(X_test_raw,y_test_raw)

[7.8550622730477357, 0.51265568487037849]

In [36]:
def combine_data_series(a,b):
    return pd.concat([a, b],axis=1)
def array_to_series(a):
    return pd.Series(a.reshape(a.shape[0]))
def combine_array_to_series(ser,arr):
    arr_ser=array_to_series(arr)
    return combine_data_series(ser,arr_ser)

In [19]:
def predict(file_path):
    df_raw = pd.read_csv(file_path,parse_dates=data.date_column)
    df = preprocess(df_raw)
    return model.predict(df,)
predictions = predict(data.test_full)

In [27]:
def write_result(X_test):
    submission_data= combine_array_to_series(X_test['click_id'],predictions)
    submission_data.columns = ['click_id', 'is_attributed']
    return submission_data

In [24]:
X_test = pd.read_csv(data.test_full,parse_dates=data.date_column)

In [26]:
X_test.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [28]:
submission_data=write_result(X_test)

In [30]:
submission_data.head()

,click_id,is_attributed
0,0,2.641286e-34
1,1,0.000000e+00
2,2,0.000000e+00
3,3,0.000000e+00
4,4,0.000000e+00


In [32]:
submission_data.to_csv('submission.csv',index=False)

In [64]:
# import keras
# from sklearn.metrics import roc_auc_score
# class PrintCallBack(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.aucs = []
#         self.losses = []
#     def on_train_end(self, logs={}):
#         return
 
#     def on_epoch_begin(self, epoch,logs={}):
#         return
 
#     def on_epoch_end(self, epoch, logs={}):
#         self.losses.append(logs.get('loss'))
#         y_pred = self.model.predict(self.validation_data[0])
#         self.aucs.append(roc_auc_score(self.validation_data[1], y_pred))
#         return 
#     def on_batch_begin(self, batch, logs={}):
#         return
 
#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))
#         return

#  class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []

#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))

# model = Sequential()
# model.add(Dense(10, input_dim=784, kernel_initializer='uniform'))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# history = LossHistory()
# model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=0, callbacks=[history])

# print(history.losses)

In [ ]:
# def f1(y_true, y_pred):
#     def precision(y_true, y_pred):
#         """Precision metric.

#         Only computes a batch-wise average of precision.

#         Computes the precision, a metric for multi-label classification of
#         how many selected items are relevant.
#         """
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#         precision = true_positives / (predicted_positives + K.epsilon())
#         return precision
#     def recall(y_true, y_pred):
#             """Recall metric.

#             Only computes a batch-wise average of recall.

#             Computes the recall, a metric for multi-label classification of
#             how many relevant items are selected.
#             """
#             true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#             possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#             recall = true_positives / (possible_positives + K.epsilon())
#             return recall

#     precision = precision(y_true, y_pred)
#     recall = recall(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))